In [ ]:
#Impport Necessary libraries

import pandas as pd
import numpy as np

In [ ]:
master_df = pd.read_csv(r"E:\DSML Projects\Credit Risk Modelling\Data\365\Loan data 2007-2014\loan_data_2007_2014.csv")

In [ ]:
df = master_df.copy()

In [ ]:
pd.options.display.max_columns = None #Display All Coluns in the Markdown
df

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

### Processing "emp_length"

In [ ]:
df['emp_length'].unique()

In [ ]:
df['emp_length'].value_counts()

In [ ]:
df['emp_length_int'] = df['emp_length'].str.replace('+ years', '')
df['emp_length_int'] = df['emp_length_int'].str.replace('< 1 year', str(0))
df['emp_length_int'] = df['emp_length_int'].str.replace('n/a',  str(0))
df['emp_length_int'] = df['emp_length_int'].str.replace(' years', '')
df['emp_length_int'] = df['emp_length_int'].str.replace(' year', '')
# We store the preprocessed ‘employment length’ variable in a new variable called ‘employment length int’,
# We assign the new ‘employment length int’ to be equal to the ‘employment length’ variable with the string ‘+ years’
# replaced with nothing. Next, we replace the whole string ‘less than 1 year’ with the string ‘0’.
# Then, we replace the ‘n/a’ string with the string ‘0’. Then, we replace the string ‘space years’ with nothing.
# Finally, we replace the string ‘space year’ with nothing.

In [ ]:
type(df['emp_length_int'][0])
# Checks the datatype of a single element of a column.

In [ ]:
df['emp_length_int'] .value_counts()

In [ ]:
df['emp_length_int'] = pd.to_numeric(df['emp_length_int'])
# Transforms the values to numeric.

In [ ]:
type(df['emp_length_int'][0])
# Checks the datatype of a single element of a column.

### Processing "earliest_cr_line"

In [ ]:
df['earliest_cr_line'].head(5)
# Displays a column.

In [ ]:
df['earliest_cr_line_date'] = pd.to_datetime(df['earliest_cr_line'], format = '%b-%y')
# Extracts the date and the time from a string variable that is in a given format.

In [ ]:
type(df['earliest_cr_line_date'][0])
# Checks the datatype of a single element of a column.

In [ ]:
pd.to_datetime('2017-12-01') - df['earliest_cr_line_date']
# Calculates the difference between two dates and times.

In [ ]:
# Assume we are now in December 2017
df['mths_since_earliest_cr_line'] = round(pd.to_numeric((pd.to_datetime('2017-12-01') - df['earliest_cr_line_date']) / np.timedelta64(1, 'M')))
# We calculate the difference between two dates in months, turn it to numeric datatype and round it.
# We save the result in a new variable.

In [ ]:
df['mths_since_earliest_cr_line'].describe()
# Shows some descriptive statisics for the values of a column.
# Dates from 1969 and before are not being converted well, i.e., they have become 2069 and similar,
# and negative differences are being calculated.

In [ ]:
df.loc[: , ['earliest_cr_line', 'earliest_cr_line_date', 'mths_since_earliest_cr_line']][df['mths_since_earliest_cr_line'] < 0]
# We take three columns from the dataframe. Then, we display them only for the rows where a variable has negative value.
# There are 2303 strange negative values.

In [ ]:
df['mths_since_earliest_cr_line'][df['mths_since_earliest_cr_line'] < 0] = df['mths_since_earliest_cr_line'].max()
# We set the rows that had negative differences to the maximum value.

In [ ]:
min(df['mths_since_earliest_cr_line'])
# Calculates and shows the minimum value of a column.

### Preprocessing "issue"

In [ ]:
df["term"].value_counts()

In [ ]:
df["term_int"] = df["term"].str.replace('months', '')

In [ ]:
df['term_int'] = pd.to_numeric(df['term_int'])

### Preprocessing "issue_d"

In [ ]:
df["issue_d"].value_counts()

In [ ]:
df["issue_date"] = df['issue_d'] = pd.to_datetime(df['issue_d'], format = '%b-%y')

In [ ]:
df["mths_since_issue_d"] =  round(pd.to_numeric((pd.to_datetime('2017-12-01') - df['issue_date']) / np.timedelta64(1, 'M')))

## Preprocessing Discrete Variables

### One Hot Encoding "grade" 

In [ ]:
pd.get_dummies(df['grade'],prefix = 'grade', prefix_sep = ':').astype(int)

In [ ]:
#OHE All the Categorical variables
df_dummies = [pd.get_dummies(df['grade'],prefix = 'grade', prefix_sep = ':').astype(int),
              pd.get_dummies(df['sub_grade'], prefix = 'sub_grade', prefix_sep = ':').astype(int),
              pd.get_dummies(df['home_ownership'], prefix = 'home_ownership', prefix_sep = ':').astype(int),
              pd.get_dummies(df['verification_status'], prefix = 'verification_status', prefix_sep = ':').astype(int),
              pd.get_dummies(df['loan_status'], prefix = 'loan_status', prefix_sep = ':').astype(int),
              pd.get_dummies(df['purpose'], prefix = 'purpose', prefix_sep = ':').astype(int),
              pd.get_dummies(df['addr_state'], prefix = 'addr_state', prefix_sep = ':').astype(int),
              pd.get_dummies(df['initial_list_status'], prefix = 'initial_list_status', prefix_sep = ':').astype(int)
             ]

In [ ]:
df_dummies = pd.concat(df_dummies, axis = 1)
df_dummies
# We concatenate the dummy variables and this turns them into a dataframe.

In [ ]:
type(df_dummies)

In [ ]:
df = pd.concat([df, df_dummies], axis = 1)

In [ ]:
df.info()

In [ ]:
df.columns.values

## Missing Value Treatment

In [ ]:
df.isnull()
# It returns 'False' if a value is not missing and 'True' if a value is missing, for each value in a dataframe.

In [ ]:
pd.options.display.max_rows = None
# Sets the pandas dataframe options to display all columns/ rows.
df.isnull().sum()

In [ ]:
pd.options.display.max_rows = 100
# Sets the pandas dataframe options to display 100 columns/ rows.

In [ ]:
# 'Total revolving high credit/ credit limit', so it makes sense that the missing values are equal to funded_amnt.
df['total_rev_hi_lim'].fillna(df['funded_amnt'], inplace=True)
# We fill the missing values with the values of another variable.

In [ ]:
df['annual_inc'].isnull().sum()

In [ ]:
df['annual_inc'].fillna(df['annual_inc'].mean(), inplace=True)

In [ ]:
df['mths_since_earliest_cr_line'].fillna(0, inplace=True)
df['acc_now_delinq'].fillna(0, inplace=True)
df['total_acc'].fillna(0, inplace=True)
df['pub_rec'].fillna(0, inplace=True)
df['open_acc'].fillna(0, inplace=True)